In [1]:
import pytest
import ipytest

ipytest.config(rewrite_asserts=True, magics=True)

__file__ = 'pytest-mock.ipynb'

## Pytest mock

When we get complex situation to test with a lot of components to set-up, we can not isolate object under test.

Imagine we get a `plane` with `engines`

In [2]:
class Airplane:
    
    def __init__(self, engines):
        self.engines = engines
        
    def is_ok(self):
        return all([engine.is_ok() for engine in self.engines])

In [3]:
class Engine:
    
    def is_ok(self):
        return len(self.get_failures()) == 0
     
                  
class PropellerEngine(Engine):

    def get_failures(self):
        failures = []
        if self.temperature > self.max_temperature:
            failures.append('temperature') 
        if self.pressure < self.min_pressure:
            failures.append('pressure')
        if self.maintenance_delay > self.max_maintenance_delay:
            failures.append('maintenance')
        return failures
        


How to test `Airplane.is_ok()` ?

In [5]:
%%run_pytest[clean] -qq


def test_is_ok():
    engines = [PropellerEngine() for i in range(4)]
    airplane = Airplane(engines)
    
    def setup_engine(engine):
        engine.max_temperature = 100
        engine.temperature = engine.max_temperature -10
        engine.min_pressure = 1900
        engine.pressure = engine.min_pressure +10
        engine.max_maintenance_delay = 365
        engine.maintenance_delay = engine.max_maintenance_delay - 100
    
    for engine in engines:
        setup_engine(engine)
    
    assert airplane.is_ok()

    

.                                                                                                                                                                                                           [100%]


It works... ***BUT***

* 1) it's **heavy**
* 2) it means you have a very important **insight** of engine internal behavior although you **just** want to test airplance
* 3) What if you change the Engine, use a jet one ?

You need to <span style="font-size: larger; font-weight: bolder;">Mock</span> engine!

## What's a mock ?

A mock is a simulated objects that **mimic** the behavior of real objects in controlled ways.

<div style="text-align: right;">Wikipedia</div>

<center><img src="https://media.giphy.com/media/C3YUQjB5agrdK/giphy.gif"></center>

## Monkeypatch

*Pytest* offer mocking features with the `monkeypatch` **fixture**.

[`monkeypatch`](https://docs.pytest.org/en/latest/monkeypatch.html) is a fixture providing a [module](http://doc.pytest.org/en/latest/_modules/_pytest/monkeypatch.html) with functions to modify object, dictionary, list in order to substitute methods, item or raise exception.

## setattr, delattr

These functions allows to modify object or class attributes, properties as methods.

In [ ]:
def set_ok(monkeypatch, engine, answer):
    monkeypatch.setattr(engine, 'is_ok', lambda: answer)

In [ ]:
%%run_pytest[clean] -qq
import pytest

def test_is_ok(monkeypatch):
    
    engines = [Engine() for i in range(4)]
    
    for engine in engines:
        set_ok(monkeypatch,engine,answer=True) # <-----
    
    airplane = Airplane(engines)
    
    assert airplane.is_ok()

**Be careful** `delattr` is misleading.

It works only on **Class**

**Doesn't work**
```python
e = Engine()
monkeypatch.delattr(e,'is_ok')
```

**Works**
```python
monkeypatch.delattr(Engine,'is_ok')
```

## setitem, delitem

Allows to modify global dictionary **for test** without altering it.

app.py
```python
CONFIG = {'data': '/data/euclid/fits/2019.09.11.data.fit', 'user': 'foo.bar'}
```

Processor.py
```python
class Processor:
    def process():
        data = app.CONFIG['data']
        ...
```


test_processor.py
```python
def test_get_process_data(create_test_data, monkeypatch):
    data_location, expected_data = create_test_data
    
    monkeypatch.setitem(CONFIG,'data',data_location) # <---------------
    
    assert processor.process() == expected_data
```    

<center>
<div class="alert alert-danger center">
    <b>Using globals</b> may be a sign a design flaw !
You have to be isolated, remember !
</div>
</center>

```python
class Processor:
    def process(data):
        ...
```

test_processor.py
```python
def test_get_process_data(create_test_data, monkeypatch):
    data_location, expected_data = create_test_data
    assert processor.process(data_location) == expected_data
```    

## setenv, delenv

Allow to modify environment value.

app.py
```python
def get_user_preferences():
    return get_file_for_user(os.getenv("USER"))
```
test_app.py
```python

def test_get_user_preferences(create_user_preferences_fixture):
    test_username, file, preferences = create_user_preferences_fixture
    
    monkeypatch.setenv('USER', test_username) # <------------
    
    assert get_user_preferences() == preferences
```    
    

## Monkeypatch is (too) simple

_Monkeypatch_ is efficient and simple, but a little too simple.

There's a lot of patterns we could use in composed tests as

* Spying calls to a components
* mocking all methods of an object
* 

TBC

# Get the power of Unittest.Mock

`unittest` is the another unit test framework.

It provides very powerfull **mocking** feature

## The Callable Box

A `Mock` is a **callable** box. 

It ever answers another `Mock`.

In [6]:
from unittest.mock import Mock

m = Mock() 

assert hasattr(m,'__call__') #<--------------- callable

m()

<Mock name='mock()' id='140643470424384'>

Use `return_value` to set a value to return.

In [ ]:
m.return_value = 'foo'
m()

Any call on any method will ever return another `Mock`

In [ ]:
r = m.one_method()
assert isinstance(r, Mock)
r()

So you could create a calling chain

In [ ]:
m = Mock()
m.n().o().p().q().r().s().t()

with a terminal returned value

In [ ]:
m = Mock()
m.n().o().p().q().r().s().t.return_value = 'foo' # <--- return_value

m.n().o().p().q().r().s().t()

`return_value` set the answer to respond and doesn't exhaust

In [ ]:
m.some_method.return_value = 'bar'

[m.some_method() for _ in range(3)] 

To set a sequence of returned values, use `side_effect`

In [ ]:
m = Mock()
m.some_method.side_effect = 'A', 'B', 'C'

[m.some_method() for _ in range(3)] 

But the next call will throw a `StopIteration` exception.

In [ ]:
try:
    m.some_method()
    assert False
except StopIteration:
    pass

`side_effect` allows to raise exceptions 

In [ ]:
m = Mock()
m.some_method.side_effect = 'A', ValueError, 'C'

assert m.some_method() == 'A'

try:
    m.some_method()   # <------------ raises ValueError
    assert False
except ValueError:
    pass

assert m.some_method() == 'C'

`Mock` constructor allow to setup complex configuration by unpacking (`**`) a dictionary

In [ ]:
props = {
    'get_maker.return_value': 'Foo',
    'get_nb_of_cylinders.return_value': 6,
    'get_max_rpm.return_value': 10000,
    'get_rpm.side_effect': [0, 1000]
}
engine = Mock(**props)

assert engine.get_maker() == 'Foo'
assert engine.get_nb_of_cylinders() == 6
assert engine.get_max_rpm() == 10000
assert engine.get_rpm() == 0
assert engine.get_rpm() == 1000

## The Recording Box

`Mock` records all calls and allow to assert some expectations about these.

In [ ]:
m = Mock()
m.some_method.return_value = 'foo'

m.some_method()

m.some_method.assert_called_once() # <-- assert been called one time

m.some_method()

assert m.some_method.call_count == 2 # <-- assert been called two times

It records the passed arguments and can compare to a `call` recipient.

In [ ]:
m.some_method('A')
m.some_method('B')

from unittest.mock import call # <------------------- call recipient

m.some_method.assert_has_calls([call('A'),call('B')])

## An others marvelous features

Take a look to [UnitTest mock official documentation](https://docs.python.org/3/library/unittest.mock.html)

In [7]:
dir(Mock())

['assert_any_call',
 'assert_called',
 'assert_called_once',
 'assert_called_once_with',
 'assert_called_with',
 'assert_has_calls',
 'assert_not_called',
 'attach_mock',
 'call_args',
 'call_args_list',
 'call_count',
 'called',
 'configure_mock',
 'method_calls',
 'mock_add_spec',
 'mock_calls',
 'reset_mock',
 'return_value',
 'side_effect']

## The Spec Feature

Creating mock could be dangerous if we lost the link with the concret classes.

As a `Mock` is a kind of _yes_ box, still answering, even if the mocked method has changed, the test is still green.



In [ ]:
class Foo:
    def some_method(self):
        return True

In [ ]:
foo = Mock()
foo.some_method.return_value = True
assert foo.some_method() == True

But if we change `Foo` ?

In [ ]:
class Foo:
    
#    def some_method(self):
#        return True
    
    def some_other_method(self):
        return 'True'

In [ ]:
# It still works :(

foo = Mock()
foo.some_method.return_value = True
assert foo.some_method() == True

`Mock(spec=<class,module>)` feature links the *specifications* to the mocked instance.

It checks the coherence between specifications and mocking.

In [ ]:
class Foo:
    def some_other_method(self):
        return 'True'

In [ ]:
foo = Mock(spec=Foo)
foo.some_method.return_value = True
assert foo.some_method() == True

## Patching

`Mock` provides powerfull patching (i.e. subtitution capability) features.

In `unittest` *patch()* is a decorator.

In [ ]:
from unittest.mock import patch

class Foo:
    def some_method(arg):
        return arg.upper()

In [ ]:
@patch('__main__.Foo')
def run_kind_of_test(foo):
    assert isinstance(foo, Mock)
    
    foo.some_method.return_value = 'BAR'
    
    assert foo.some_method() == 'BAR'
    
run_kind_of_test()

Patching is very useful to substitute external or built-in modules

`patch` substitutes to the used symbole address by the target another object.

The **tricky thing** is to locate the symbol address.

<div class="alert alert-info text-center">We have to patch() the object where it is looked up.</div>

In [8]:
from datetime import datetime, date


assert __name__ == "__main__" #<----------- We are in the __main__ namespace

def are_we_in_21_century():
    return date.today().year <= 2000  # <-- Here 'date' is the symbole used from __main__

@patch('__main__.date') #<------------------ "__main__.date" points 'date' in are_we_in_21_century
def run_kind_of_test(mocked_date):
 
    import datetime  #<---------------------- Use a different namespace to get a real date
    mocked_date.today.return_value = datetime.date(1975,5,31)
    
    assert not isinstance(date.today(), Mock)
    
    assert are_we_in_21_century()
    
run_kind_of_test()

NameError: name 'patch' is not defined

<div class="alert alert-warning">The following is for advanced users !!</div>

When we make an `import`, we put a **symbol** into a local table of symbols.

module_A.py
```python
import sys

current_module == sys.modules['module_A']

assert 'date' not in current_module.local_variable

from datetime import date

assert 'date' in current_module.local_variable # <--- now a "module_A.date" symbol exists
```

This is this **"module_A.date"** path we have to use in our `patch` **because** this is this symbole that would used by python runtime.

```python
from datetime import date

print(date.today())
```

Python runtime acts like 


* Oh I have a `date` symbole to use.
* I lookup it up in my *table of symbols* for *module_A*
* I used `module_A.date` as key
* I retrieve a value ( `module_A.date` &rarr;  value)
* I apply `today()` on this value.

## And what about pytest ?

# pytest-mock

[`pytest-mock`](https://pypi.org/project/pytest-mock/) is a pytest-plugin  
bringing the `unittest` very powerfull `mock` library  
and others mocking tools  
throught `mocker` fixture.

## Mocker fixture


## Create a pure Mock object

In [ ]:
class Car:
    def __init__(self,engine):
        self.engine = engine
        
    def is_started(self):
        return self.engine.is_started()

In [ ]:
%%run_pytest[clean] -qq

import pytest
from pytest_mock import mocker #<----------------- import mocker


def test_car_is_started(mocker):
    
    engine = mocker.Mock() #<--------------------- creates a Mock
    engine.is_started.return_value = True # <------create a sub-Mock that can return value
    
    car = Car(engine)
    
    assert car.is_started() ==True


To set a **sequence** of returned values use `Mock.side_effect`

In [ ]:
%%run_pytest[clean] -qq

def test_car_is_started(mocker):
    
    engine = mocker.Mock() 
    engine.is_started.side_effect = True, False, True
    
    car = Car(engine)
    
    assert car.is_started() ==True
    assert car.is_started() ==False
    assert car.is_started() ==True


To set a **sequence** of returned values **or exception** use `Mock.side_effect`

In [ ]:
%%run_pytest[clean] -qq

def test_car_is_started(mocker):
    
    engine = mocker.Mock() 
    engine.is_started.side_effect = True, ValueError, True 
    
    car = Car(engine)
    
    assert car.is_started() ==True
    with pytest.raises(ValueError): # <------ ValueError is raised
        car.is_started()            #
    assert car.is_started() ==True


`Mock` constructor allow to setup complex Mock by unpacking (`**`) a dictionary

* Mock.some_method.**return_value** &rarr; next value to return when _some_method_ will be called. 
* Mock.some_method.**side_effect** &rarr; **sequence** of value to return **or Exception** to raise when _some_method_ will be called. 

## Mocker.patch

As `Mocker` fixture exist, for patching we don't use *decorator* 

In [ ]:
from datetime import datetime, date

class Menu:
    def __init__(self, menu_of_week):
        self.menu_of_week = menu_of_week
        
    def get_today_menu(self):
        today = date.today().isoweekday() #<--------------- '__main__.date' symbol
        return self.menu_of_week[today]

In [ ]:
%%run_pytest[clean] -qq

def test_menu(mocker):
    # given
    mocker.patch('__main__.date')
    date.today().isoweekday.return_value = 1
    
    menu = Menu(['bread', 'ham', 'egg', 'fish', 'cake'])

    
    assert menu.get_today_menu() == 'ham'

## Mocker.spy

A nice feature is just tracking call of real objects.

In [13]:
class Foo:

    def bar(self):
        pass

    def do_it(self, value):
        for _ in range(value):
            self.bar()

In [14]:
%%run_pytest[clean] -qq

def test_how_many_bar_callsb(mocker):
    foo = Foo()
    mocker.spy(foo, 'bar')
    
    foo.do_it(3)
    
    assert foo.bar.call_count == 3
    

.                                                                                                                                                                                                           [100%]


# Readings

* [Mock official documentation](https://docs.python.org/3/library/unittest.mock.html)
* [pytest-mock](https://pypi.org/project/pytest-mock/)
* [Mock cookbook](https://chase-seibert.github.io/blog/2015/06/25/python-mocking-cookbook.html)
* [Mock Real Python](https://realpython.com/python-mock-library/)